In [1]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
# Import necessary dataframes
nj_current_assignment = pd.read_csv('nj_longlat_unfairmap.csv')
nj_current_assignment

nj_precinct_data = pd.read_csv('precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Total', 'Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]

nj_current_assignment = nj_current_assignment.set_index('GEOID20', drop= False)
nj_precinct_data = nj_precinct_data.set_index('GEOID20', drop = False)

nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)
nj_contiguity.columns = ['Precinct','Neighbors']

#updates precinct data to seed map districts
for index, short in nj_current_assignment.iterrows():
    nj_precinct_data.loc[short.name, 'District'] = short['District']
    
#used
pop_DF = pd.DataFrame(columns =['District', 'Pop'])
total = 0
for i in range(1,13):
    idistrict =nj_precinct_data[nj_precinct_data['District'] == i]
    pop = sum(idistrict['Total_2020_Total'])
    pop_DF.loc[len(pop_DF)] = (i, pop)
    
pop_DF =pop_DF.set_index('District', drop = False)

In [3]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [4]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [5]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

In [6]:
def isDistrictContiguous(district_num, assignment, contiguity_list, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    return nx.is_connected(district_graph)

In [7]:
#Create modified Corey GeoDistricts as seed map
#Use once to create unfair seed map

#Prepare Shapely
# shpfile = 'nj_vtd_2020_bound.shp'
# dbffile = 'nj_vtd_2020_bound.dbf'
# shxfile = 'nj_vtd_2020_bound.shx'


# shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
# nj_precinct_boundaries={}
# for sr in shpfile.iterShapeRecords():
#     geom = sr.shape # get geo bit
#     rec = sr.record # get db fields
#     nj_precinct_boundaries[rec[3]]=geom


# for index, row in nj_longlat_assignment.iterrows():
#     try:
#         if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
#             centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
#         elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
#             centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
#         else:
#             print(shape(nj_precinct_boundaries[row['GEOID20']]).type)
#             pass
#         if centroid.x <= -74.52:
#             if centroid.y <= 39.7:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
#             elif centroid.y <= 39.9:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
#             elif centroid.y <= 40.5:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
#             elif centroid.y <= 40.7:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
#             elif centroid.y <= 40.9:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
#             else:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
#         else:
#             if centroid.y <= 39.7:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
#             elif centroid.y <= 39.9:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
#             elif centroid.y <= 40.5:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
#             elif centroid.y <= 40.7:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
#             elif centroid.y <= 40.9:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
#             else:
#                 nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
#     except KeyError: 
#         pass

# nj_longlat_assignment.to_csv('nj_longlat_unfairmap.csv',index=False)

# print(isDistrictContiguous(1,nj_longlat_assignment,nj_contiguity))
# print(isDistrictContiguous(2,nj_longlat_assignment,nj_contiguity))

# print('Populations: ' + str(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 2)))

In [8]:
def update_districts(nj_precinct_data):
    district_pops = []

    for i in range(1, 13):
        idistrict = nj_precinct_data[nj_precinct_data['District'] == i]
        pop = idistrict['Total_2020_Total'].sum()
        district_pops.append((i, pop))

    # Create a fresh DataFrame
    pop_DF = pd.DataFrame(district_pops, columns=['District', 'Pop'])
    pop_DF.set_index('District', inplace=True)

    return pop_DF

In [9]:
#2020 Fair district population balances
#nj_current_assignment
# total NJ pop 9,288,994
# Target district pop = 774,083
# +-3800 to be within 1% of target district population
# For testing purposes variance set to 30000

def check_all_equalized(pop_DF):
    
    pop_DF = update_districts(nj_precinct_data)
    
    lower_d =pop_DF['Pop'].min()
    higher_d =pop_DF['Pop'].max()
    target_var = 30000
    diff = higher_d - lower_d
    
    print(str(higher_d) + " - " + str(lower_d) + " = " + str(diff) + " <= ? " + str(target_var))
    if((diff) <= target_var):
        return True
    else:
        return False
    

In [10]:
# # Update to Unfair seed District population balances
# for index, i in nj_current_assignment.iterrows():
#     nj_precinct_data.loc[i.name, 'District'] = i.District
# pop_DF = update_districts(nj_precinct_data)


# total = 0
# for i in range(1,13):
#     idistrict =nj_precinct_data[nj_precinct_data['District'] == i]
#     pop = sum(idistrict['Total_2020_Total'])
#     total = total + pop
#     pop_DF.loc[len(pop_DF)] = (i, pop)
# print('Total: ' + str(total))
# pop_DF =pop_DF.set_index('District', drop = False)
# print(pop_DF)
# print(total/12)

In [11]:
def find_neighbors(D1_list, target_district):
    D1_border_precincts = []
    
    for precinct in D1_list:
        neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
        for neighbor in neighbors:
            if nj_current_assignment.loc[nj_current_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==target_district:
                D1_border_precincts.append(precinct)
                break
    
    D1_border_precincts_sorted = sorted(D1_border_precincts,
    key=lambda geo: nj_precinct_data.loc[nj_precinct_data['GEOID20'] == geo, 'Total_2020_Total'].values[0],reverse=True)
    
    return D1_border_precincts

In [14]:
def pop_rebalance(pop_DF, nj_precinct_data, nj_current_assignment):
    
    district_neighbors = 0
    target_max_pop = 774082 + 30000
    target_min_pop = 774082 - 30000
    i = 1
    
    
    while not check_all_equalized(pop_DF):
        largest = pop_DF['Pop'].idxmax()

        #Identify neighboring districts to our largest district from our 2 column, 6 row seed grid
        if largest ==12:
            district_neighbors = 11
        elif largest ==11:
            district_neighbors = 9
        elif largest ==2:
            district_neighbors = 4
        elif largest ==1:
            district_neighbors = 2
        elif largest %2 == 0: 
            district_neighbors = largest + 2
        else:
            district_neighbors = largest -2
            
        print(str(largest) + ": " + str(district_neighbors))
        print('Cycle ' + str(i))
        
        D1_list = nj_current_assignment[nj_current_assignment['District']== largest]['GEOID20']
        D1_border_precincts = find_neighbors(D1_list, district_neighbors)
        
        
#         pop_DF.copy()
#         while pop_DF.loc[largest].Pop > target_max_pop:
#             print('1: ' +str(pop_DF.loc[largest].Pop)+ ' > ' + str(target_max_pop))
#             D1_border_precincts = find_neighbors(D1_list, district_neighbors)
#             for pre in D1_border_precincts:
#                 if pop_DF.loc[largest].Pop > target_max_pop:
#                     nj_precinct_data.loc[pre,'District'] = district_neighbors
#                     nj_current_assignment.loc[pre,'District'] = district_neighbors
#                     if not isDistrictContiguous(district_neighbors, nj_current_assignment, nj_contiguity):
#                         nj_precinct_data.loc[pre,'District'] = largest
#                         nj_current_assignment.loc[pre,'District'] = largest
#                     elif not isDistrictContiguous(largest, nj_current_assignment, nj_contiguity):
#                         nj_precinct_data.loc[pre,'District'] = largest
#                         nj_current_assignment.loc[pre,'District'] = largest
#                     pop_DF= update_districts(nj_precinct_data)
#         i = i +1
        
    
        moved = False
        for pre in D1_border_precincts:
            if pop_DF.loc[largest].Pop <= target_max_pop:
                break

            nj_precinct_data.loc[pre, 'District'] = district_neighbors
            nj_current_assignment.loc[pre, 'District'] = district_neighbors
            if not (isDistrictContiguous(district_neighbors, nj_current_assignment, nj_contiguity) and
                isDistrictContiguous(largest, nj_current_assignment, nj_contiguity)):
                # Revert if contiguity is broken
                nj_precinct_data.loc[pre, 'District'] = largest
                nj_current_assignment.loc[pre, 'District'] = largest
            else:
                moved = True  # At least one valid move occurred
        if moved:
            # Only update population if any precinct was successfully moved
            pop_DF = update_districts(nj_precinct_data)
        else:
            print('Skipping')
            break  # Prevent infinite loop if stuck

        i += 1

In [ ]:
nj_precinct_data, nj_current_assignment = pop_rebalance(pop_DF, nj_precinct_data, nj_current_assignment)

2439952 - 93401 = 2346551 <= ? 30000
10: 12
Cycle 1
2359116 - 93401 = 2265715 <= ? 30000
10: 12
Cycle 2
2256150 - 93401 = 2162749 <= ? 30000
10: 12
Cycle 3
2153693 - 93401 = 2060292 <= ? 30000
10: 12
Cycle 4
2043885 - 93401 = 1950484 <= ? 30000
10: 12
Cycle 5
1934016 - 93401 = 1840615 <= ? 30000
10: 12
Cycle 6
1803476 - 93401 = 1710075 <= ? 30000
10: 12
Cycle 7
1700071 - 93401 = 1606670 <= ? 30000
10: 12
Cycle 8
1792152 - 93401 = 1698751 <= ? 30000
12: 11
Cycle 9
1769560 - 93401 = 1676159 <= ? 30000
12: 11
Cycle 10
1750686 - 93401 = 1657285 <= ? 30000
12: 11
Cycle 11
1727958 - 93401 = 1634557 <= ? 30000
12: 11
Cycle 12
1692619 - 93401 = 1599218 <= ? 30000
12: 11
Cycle 13
1659449 - 93401 = 1566048 <= ? 30000
12: 11
Cycle 14
1610287 - 93401 = 1516886 <= ? 30000
12: 11
Cycle 15
1576379 - 93401 = 1482978 <= ? 30000
10: 12
Cycle 16
1680597 - 93401 = 1587196 <= ? 30000
12: 11
Cycle 17
1619355 - 93401 = 1525954 <= ? 30000
12: 11
Cycle 18
1564653 - 93401 = 1471252 <= ? 30000
12: 11
Cycle 19
15

In [ ]:
# nj_current_assignment_reset = nj_current_assignment.reset_index(drop =True)
# nj_precinct_data_reset = nj_precinct_data.reset_index(drop = True)

# # Merge on GEOID20 to align district with population
# merged = nj_current_assignment_reset.merge(
#     nj_precinct_data_reset[['GEOID20', 'Total_2020_Total']],
#     on='GEOID20',
#     how='left')

# # Group by district and sum population
# dis_pops = merged.groupby('District')['Total_2020_Total'].sum().to_frame(name='Pop')
# print(dis_pops)

In [ ]:
#nj_current_assignment.to_csv('nj_longlat_unfairmap_maybeFinal.csv',index=False)